In [2]:
import pandas as pd
from plotly.offline import iplot
import plotly.graph_objs as go
import numpy as np
from sklearn import metrics
import math
import matplotlib.pyplot as plt
from collections import defaultdict
from functools import partial
from itertools import repeat
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation

Using TensorFlow backend.


In [1]:
workDir = "/Users/brandonmanley/Documents/nBody/"

#Import data
fname = workDir + "data/brutusSim/3body/brutus10_4_3.csv"
df = pd.read_csv(fname)
with pd.option_context('mode.use_inf_as_null', True):
    df = df.dropna()
dfShuffle = shuffle(df,random_state=42)
df.head()

NameError: name 'pd' is not defined

In [4]:
i_col = ["m1","m2","m3","x1", "x2", "x3", "y1", "y2", "y3",
		"dx1","dx2","dx3","dy1","dy2","dy3","tEnd"]
o_col = ["x1tEnd", "x2tEnd", "x3tEnd", "y1tEnd", "y2tEnd", "y3tEnd",
		"dx1tEnd", "dx2tEnd", "dx3tEnd", "dy1tEnd", "dy2tEnd", "dy3tEnd"]

X1 = df.as_matrix(columns=i_col)
y1 = df.as_matrix(columns=o_col)

X_train,X_test,y_train,y_test = train_test_split(X1,y1, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape,y_test.shape)

/home/llavezzo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/home/llavezzo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



(742632, 16) (742632, 12)
(185658, 16) (185658, 12)


In [5]:
for i in y_train[0]:
    print(i)

4.119593923
6.427622086
-9.418013716
10.97865132
6.5292792010000005
-16.0771539
-0.20135184399999997
0.909095652
-1.554035885
-0.178778333
1.670345187
-2.092791147


In [6]:
network = models.Sequential()
network.add(layers.Dense(16,activation='relu',input_dim=16))
for i in range(10):
    network.add(layers.Dense(128,activation='relu'))
network.add(layers.Dense(12,activation='relu'))
network.compile(optimizer='adam',loss='mean_squared_logarithmic_error',metrics=['accuracy'])
network.save_weights(workDir + '/weights/model_init.h5')

In [ ]:
network.load_weights(workDir + '/weights/model_init.h5')
history = network.fit(X_train,y_train,
                      epochs=300,
                      batch_size=1000,
                      validation_data=(X_test,y_test), 
                      verbose = 1)